In [1]:
#to open cred files
import os

#Data transformation
import pandas as pd

#bigquery
from google.cloud import bigquery

#FIREBASE
import firebase_admin

from firebase_admin import credentials
from firebase_admin import firestore

from dotenv import load_dotenv
load_dotenv('/Users/daniel/OAN/credentials/miong/.env')

print('beginning etl of accounting ... ')

#OPEN CONECTION TO FIREBASE
filename=os.environ['FIREBASE_MIONG_FILENAME']

beginning etl of accounting ... 


In [2]:
## OPEN CONECTION TO BIGQUERRY
bgq_client = bigquery.Client()
dataset_id = "{}.oan".format(bgq_client.project)
dataset = bigquery.Dataset(dataset_id)

In [3]:
cred = credentials.Certificate(filename)
firebase_admin.initialize_app(cred)

db = firestore.client()

In [4]:
OAN_account = "5Tv2u4n8BReebmKUNIuN"

In [5]:
#get all the collection of OAN
col_query = db.collection('accountingItems').where(u'context.account', u'==',OAN_account).stream()

In [6]:
#save it in a list of dictionaries
accounts = []
for acc in col_query:
    accounts.append(acc.to_dict())

In [153]:
for i,acc in enumerate(accounts):
    if acc['context']['id'] == '4TRVpm2unWZMfYyPGbtQ':
        a=acc
        b=i

In [154]:
a

{'tags': ['spain', 'shopify'],
 'vat': 21,
 'originAccountingAccount': '3deqBmrZ1iJkAwMmxSOp',
 'targetIntervention': 'WYTJNewInibo0kumno7l',
 'concept': 'Venta Shopify',
 'responsible': 'lwUfGzSzoJuoUCIY83Co',
 'vatAmount': 12.14,
 'baseAmount': 69.95,
 'context': {'createdByType': 'shopify',
  'id': '4TRVpm2unWZMfYyPGbtQ',
  'createdBy': 'oXJJEfAEPxFYtdJ2pnaU',
  'createdAt': '2022-01-06T13:29:40.112Z',
  'account': '5Tv2u4n8BReebmKUNIuN'},
 'targetProject': '0DmODGTOEiM5lg9SGx0J',
 'files': [],
 'description': 4345036275811,
 'executedAt': '2022-01-06T13:29:40.112Z',
 'amount': 90,
 'code': 93,
 'type': 'income',
 'targetAccountingAccount': 'iA9Pzv2CImjItzwCaQv0'}

In [155]:
b

320

In [7]:
mappedFields = {
    'concept': 'concept',
    'vat': 'vat',
    'description':'description',
    'amount':'amount',
    'execution_date': 'executedAt',
    'user_in_charge': 'responsible',
    'account_id': 'originAccountingAccount',
    'code':'code',
    'type':'type',
    'target_id': 'targetAccountingAccount',
    'vat_amount': 'vatAmount'   
}
mappedContextFields = {
    'creation_date': 'createdAt',
    'creator_user':'createdBy',
    'id':'id',
    'origin': 'createdByType'
}

In [8]:
def convertData(acc):
    accItem = dict()
    accKeys = acc.keys()
    tag = None
    if acc["tags"]:
        tag = "spain" if "spain" in acc["tags"] else "benin" if "benin" in acc["tags"] else ""
    accItem["place"] = tag
    if 'files' in accKeys:
        if acc['files']:
            accItem['receipt'] = acc['files'][0]['downloadUrl']
        else:
            accItem['receipt'] = None
    for p in ['project','phase','intervention']:
        if 'origin'+p[0].capitalize()+p[1:] in accKeys:
            if acc['origin'+p[0].capitalize()+p[1:]] != '':
                accItem[p] = acc['origin'+p[0].capitalize()+p[1:]]
            elif 'target'+p[0].capitalize()+p[1:] in accKeys:
                accItem[p] = acc['target'+p[0].capitalize()+p[1:]]
            else:
                accItem[p] = ''
        elif 'target'+p[0].capitalize()+p[1:] in accKeys:
                accItem[p] = acc['target'+p[0].capitalize()+p[1:]]
        else:
            accItem[p] = ''
    for key in mappedFields:
        accItem[key] = acc[mappedFields[key]] if mappedFields[key] in accKeys else ""
    for key in mappedContextFields:
        accItem[key] = acc['context'][mappedContextFields[key]]
    return accItem

In [9]:
convAccounts = []
for i,acc in enumerate(accounts):
    convAccounts.append(convertData(acc))

In [10]:
#convert to dataframe
accounting = pd.DataFrame(convAccounts)

In [13]:
accounting['creation_date'] = pd.to_datetime(accounting['creation_date'],utc=True)
accounting['execution_date'] = pd.to_datetime(accounting['execution_date'],utc=True)

In [14]:
#and the order whe want to upload to bigquery
df = accounting[['concept', 'place', 'creation_date', 'vat', 'description', 'amount',
       'id', 'phase', 'creator_user', 'execution_date',
       'user_in_charge', 'origin', 'account_id', 'project', 'code', 'type',
       'intervention', 'target_id', 'vat_amount', 'receipt']]

In [17]:
accounting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4190 entries, 0 to 4189
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   place           4187 non-null   object             
 1   receipt         542 non-null    object             
 2   project         4190 non-null   object             
 3   phase           1468 non-null   object             
 4   intervention    4190 non-null   object             
 5   concept         4190 non-null   object             
 6   vat             3942 non-null   object             
 7   description     4091 non-null   object             
 8   amount          4190 non-null   float64            
 9   execution_date  4190 non-null   datetime64[ns, UTC]
 10  user_in_charge  4190 non-null   object             
 11  account_id      4190 non-null   object             
 12  code            4190 non-null   int64              
 13  type            4190 non-null   o

In [18]:
table_id = "{}.accounting".format(dataset.dataset_id)
proj = bgq_client.project

df.to_gbq(table_id,proj,if_exists='replace')

print('done.')

1it [00:14, 14.77s/it]

done.
